In [2]:
import argparse
import torch
import numpy as np
import slayerSNN as snn
from pathlib import Path

path = "/home/tasbolat/some_python_examples/data_VT_SNN/"
tact = torch.load(Path(path) / "tact.pt")

In [3]:
tact.shape

torch.Size([400, 78, 2, 325])

In [ ]:
# parser = argparse.ArgumentParser()

# parser.add_argument("--path", help="Path", required=True)
# parser.add_argument("--count", type=int, help="count", required=True)

# args = parser.parse_args()

In [19]:
def convert_to_rectangle(a):
    img_depth_list = []
    for i in range(a.shape[-1]):
        img_depth_list.append([
            [a[3,i], (a[3,i]+a[10,i]+a[6,i])/3, a[10,i], (a[10,i]+a[29,i]+a[14,i])/3 , a[24,i], (a[24,i]+a[29,i]+a[34,i])/3, a[34,i]],
            [(a[3,i]+a[1,i]+a[6,i])/3   , a[6,i], (a[6,i]+a[10,i]+a[11,i]+a[17,i]+a[19,i])/5, a[17,i], (a[24,i]+a[28,i]+a[29,i]+a[17,i]+a[19,i])/5 , a[29,i], (a[29,i]+a[34,i]+a[36,i])/3],
            [a[1,i], (a[1,i]+a[6,i]+a[8,i]+a[11,i]+a[15,i])/5, a[11,i], a[19,i], a[28,i], (a[22,i]+a[28,i]+a[29,i]+a[36,i]+a[32,i])/5, a[36,i]],
            [(a[0,i]+a[1,i]+a[5,i]+a[8,i])/4, a[8,i], a[15,i], (a[11,i]+a[15,i]+a[14,i]+a[20,i]+a[26,i]+a[22,i]+a[28,i]+a[19,i])/8, a[22,i], a[32,i], (a[32,i]+a[33,i]+a[38,i]+a[36,i])/4],
            [a[0,i], a[5,i], a[14,i], a[20,i], a[26,i], a[33,i], a[38,i]],
            [(a[0,i]+a[2,i]+a[5,i]+a[9,i])/4, a[9,i], a[16,i], (a[14,i]+a[16,i]+a[13,i]+a[21,i]+a[27,i]+a[23,i]+a[26,i]+a[20,i])/8, a[23,i], a[30,i], (a[33,i]+a[30,i]+a[38,i]+a[37,i])/4],
            [a[2,i], (a[2,i]+a[7,i]+a[9,i]+a[13,i]+a[16,i])/5, a[13,i], a[21,i], a[27,i], (a[23,i]+a[27,i]+a[31,i]+a[37,i]+a[30,i])/5, a[37,i]],
            [(a[2,i]+a[7,i]+a[4,i])/3 , a[7,i], (a[12,i]+a[13,i]+a[7,i]+a[18,i]+a[21,i])/5, a[18,i], (a[21,i]+a[18,i]+a[25,i]+a[31,i]+a[27,i])/5, a[31,i], (a[31,i]+a[35,i]+a[37,i])/3],
            [a[4,i], (a[4,i]+a[7,i]+a[18,i])/3   , a[12,i], (a[12,i]+a[18,i]+a[25,i])/3, a[25,i], (a[31,i]+a[25,i]+a[35,i])/3, a[35,i]]          
        ])
    return img_depth_list

In [34]:
all_data_l = np.zeros([400, 2, 9, 7, 325])
all_data_r = np.zeros([400, 2, 9, 7, 325])

In [36]:
all_data_l[k].shape

(2, 9, 7, 325)

In [37]:

for k in range(400):
    print(k)
    sample_file = tact[k,...]
    channel_list_left = []
    channel_list_right = []
    for channel in range(2):
        a = sample_file[:,channel,:]
        img_depth_right = convert_to_rectangle(a[:39])
        img_depth_left = convert_to_rectangle(a[39:])
        img_depth = img_depth_right + img_depth_left
        channel_list_left.append(img_depth_left)
        channel_list_right.append(img_depth_right)
        
    ri_left = np.array(channel_list_left)
    ri_left[ri_left > 0.5] = 1
    ri_left[ri_left <= 0.5] = 0
    ri_left = ri_left.swapaxes(2,1).swapaxes(2,3).astype(float)
    
    #ri_left = torch.FloatTensor(ri_left)
    all_data_l[k] = ri_left
    
    ri_right = np.array(channel_list_right)
    ri_right[ri_right > 0.5] = 1
    ri_right[ri_right <= 0.5] = 0
    
    ri_right = ri_right.swapaxes(2,1).swapaxes(2,3).astype(float)
    #ri_right = torch.FloatTensor(ri_left)
    all_data_r[k] = ri_right

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [38]:
all_data_r = torch.FloatTensor( all_data_r.astype(float) )
all_data_l = torch.FloatTensor( all_data_l.astype(float) )

In [39]:
all_data_r.shape, all_data_l.shape

(torch.Size([400, 2, 9, 7, 325]), torch.Size([400, 2, 9, 7, 325]))

In [41]:
save_path = path + "tactile_rectangular/"

In [43]:
torch.save(all_data_r, save_path + 'tac_right.pt')
torch.save(all_data_l, save_path + 'tac_left.pt')